In [14]:
# Подключаем необходимые библиотеки
import pandas as pd
from datetime import datetime
import re
from dateutil.relativedelta import relativedelta

In [15]:
df_financial = pd.read_csv('financial_data.csv')
df_prolong = pd.read_csv('prolongations.csv')

In [16]:
def parse_number(x):
    if isinstance(x, str):
        x = x.strip().lower()
        if x in ['стоп', 'stop', 'end']:
            return 'стоп'
        if x in ['в ноль']:
            return 'в ноль'
        x = x.replace(' ', '').replace(',', '.')
        try:
            return float(x)
        except:
            return 0
    elif pd.isna(x):
        return 0
    return x


In [17]:
# # Очистка числовых столбцов
month_cols = [col for col in df_financial.columns if re.match(r'[А-Яа-я]+\s\d{4}', col)]
for col in month_cols:
    df_financial[col] = df_financial[col].apply(parse_number)

merged = df_prolong.merge(df_financial, on='id', how='left')

In [18]:
months = [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
]
month_name_to_number = {name.lower(): i + 1 for i, name in enumerate(months)}
month_number_to_name = {i + 1: name for i, name in enumerate(months)}

In [19]:
# Функция для получения значения с учётом "в ноль"
def get_adjusted_value(row, target_month, later_months, previous_month):
    value = row.get(target_month)
    if value == 'стоп':
        return None  # не учитываем проекты со статусом "стоп"
    if value == 'в ноль':
        only_zeros = all(
            row.get(m, 0) in [0, 'в ноль'] for m in later_months
        )
        if only_zeros:
            return row.get(previous_month, 0)
        else:
            return 0
    return value if value != 'в ноль' else 0


In [20]:

def calculate_coefficients(df, current_month):
    coefficients = []

    try:
        month_str, year_str = current_month.strip().split()
        current_dt = datetime(int(year_str), month_name_to_number[month_str.lower()], 1)
    except Exception as e:
        print(f"Ошибка парсинга месяца {current_month}: {e}")
        return pd.DataFrame()

    # Определение нужных месяцев
    prev_dt = current_dt - relativedelta(months=1)  # апрель, если текущий — май
    prev2_dt = current_dt - relativedelta(months=2)  # март, если текущий — май

    prev_month = f"{month_number_to_name[prev_dt.month]} {prev_dt.year}"
    prev2_month = f"{month_number_to_name[prev2_dt.month]} {prev2_dt.year}"
    current_month_str = f"{month_number_to_name[current_dt.month]} {current_dt.year}"

    later_months_from_prev = [
        f"{month_number_to_name[(prev_dt.month + i - 1) % 12 + 1]} {prev_dt.year + ((prev_dt.month + i - 1) // 12)}"
        for i in range(1, 13)
    ]
    later_months_from_prev2 = [
        f"{month_number_to_name[(prev2_dt.month + i - 1) % 12 + 1]} {prev2_dt.year + ((prev2_dt.month + i - 1) // 12)}"
        for i in range(1, 13)
    ]

    for account, group in df.groupby('Account'):
        k1_num = 0
        k1_den = 0
        k2_num = 0
        k2_den = 0

        for _, row in group.iterrows():
            # Пролонгация в 1-й месяц после окончания
            end_val = get_adjusted_value(row, prev_month, later_months_from_prev, '')
            next_val = get_adjusted_value(row, current_month_str, [], prev_month)
            if end_val is not None and isinstance(end_val, (int, float)) and end_val > 0:
                k1_num += next_val if isinstance(next_val, (int, float)) else 0
                k1_den += end_val

            # Пролонгация во 2-й месяц после окончания
            end2_val = get_adjusted_value(row, prev2_month, later_months_from_prev2, '')
            skip_val = get_adjusted_value(row, prev_month, later_months_from_prev, prev2_month)
            second_val = get_adjusted_value(row, current_month_str, [], prev_month)

            if end2_val is not None and isinstance(end2_val, (int, float)) and end2_val > 0 and (not skip_val or skip_val == 0):
                k2_num += second_val if isinstance(second_val, (int, float)) else 0
                k2_den += end2_val

        k1 = round(k1_num / k1_den, 2) if k1_den else None
        k2 = round(k2_num / k2_den, 2) if k2_den else None

        coefficients.append({
            'Account': account,
            'K1': k1,
            'K2': k2
        })

    return pd.DataFrame(coefficients)

In [26]:
# Список месяцев
months_to_analyze = sorted(set(df_prolong['month']))

# Расчёт коэффициентов
results = []
for month in months_to_analyze:
    for manager in merged['AM'].dropna().unique():
        manager_data = merged[merged['AM'] == manager]
        coeffs = calculate_coefficients(manager_data, month)

        if not coeffs.empty:
            avg_k1 = coeffs['K1'].mean(skipna=True)
            avg_k2 = coeffs['K2'].mean(skipna=True)

            results.append({
                'Месяц': month,
                'Менеджер': manager,
                'K1': avg_k1,
                'K2': avg_k2
            })


In [27]:
final_df = pd.DataFrame(results)

# Группировки
final_df['Год'] = final_df['Месяц'].str.extract(r'(\d{4})')
yearly_report = final_df.groupby(['Год', 'Менеджер'])[['K1', 'K2']].mean().reset_index()
overall_report = final_df.groupby('Менеджер')[['K1', 'K2']].mean().reset_index()

overall_summary = pd.DataFrame({
    'Менеджер': ['Весь отдел'],
    'K1': [final_df['K1'].mean(skipna=True)],
    'K2': [final_df['K2'].mean(skipna=True)]
})


# Сохранение
final_df.to_csv('prolongation_report_fixed.csv', index=False)
yearly_report.to_csv('yearly_report.csv', index=False)
overall_report.to_csv('overall_report.csv', index=False)
overall_summary.to_csv('department_summary.csv', index=False)

print("Все отчёты успешно сформированы и сохранены в CSV!")
print("Пример данных:")
print(final_df.head())

Все отчёты успешно сформированы и сохранены в CSV!
Пример данных:
         Месяц                       Менеджер    K1   K2   Год
0  август 2023   Васильев Артем Александрович   NaN  NaN  2023
1  август 2023      Михайлов Андрей Сергеевич   NaN  NaN  2023
2  август 2023  Соколова Анастасия Викторовна  0.67  NaN  2023
3  август 2023        Иванова Мария Сергеевна   NaN  NaN  2023
4  август 2023    Попова Екатерина Николаевна   NaN  0.0  2023
